In [2]:
import json
import base64
import requests
from PIL import Image
from datetime import datetime
from cryptography.fernet import Fernet

def generate_key():
    return Fernet.generate_key()

# Encrypt a message using symmetric encryption
def encrypt_message(message, key):
    cipher = Fernet(key)
    return cipher.encrypt(message.encode())

def decrypt_message(ciphertext, key):
    cipher = Fernet(key)
    return cipher.decrypt(ciphertext).decode()

def get_current_location():
    response = requests.get('https://ipinfo.io/json')
    data = response.json()
    city = data.get('city', '')
    region = data.get('region', '')
    country = data.get('country', '')
    loc = data.get('loc', '')  # latitude,longitude
    return {
        'city': city,
        'region': region,
        'country': country,
        'coordinates': loc
    }

## LSB Encoder

In [3]:
"""
Encode a secret message into an image using LSB stenography.

This function encrypts a given secret message with a specified key, encodes
the encrypted message into a URL-safe Base64 string, and embeds it into the
least significant bits of the image's pixels. The modified image is saved to
the specified output path.

Parameters:
    img_path (str): The file path to the input image.
    secret_message (str): The message to be encrypted and embedded.
    key (str): The encryption key used to encrypt the secret message.
    output_path (str): The file path to save the encoded image.

Returns:
    None
"""
def lsb_encode(img_path, secret_message, key, output_path):
    # Encrypt message first
    enc_bytes = encrypt_message(secret_message, key)
    # For embedding, encode bytes to base64 string (to ensure printable chars)
    enc_b64 = base64.urlsafe_b64encode(enc_bytes).decode()
    # Add a null-terminator for extraction
    enc_b64 += chr(0)

    img = Image.open(img_path)
    encoded = img.copy()
    width, height = img.size
    data = ''.join(f"{ord(c):08b}" for c in enc_b64)
    idx = 0

    for y in range(height):
        for x in range(width):
            pixel = list(img.getpixel((x, y)))
            for n in range(3):  # R, G, B
                if idx < len(data):
                    pixel[n] = pixel[n] & ~1 | int(data[idx])
                    idx += 1
            encoded.putpixel((x, y), tuple(pixel))
            if idx >= len(data):
                encoded.save(output_path)
                return

## LSB Decoder

In [4]:
"""
Decode a hidden message from an image using LSB steganography and decrypt it.

This function extracts the least significant bits from each pixel of the image
at the given path to reconstruct a binary message. The message is assumed to be
encoded in base64 and is terminated by a null character. The base64-encoded
message is then decoded to bytes and decrypted using the provided key.

Args:
    img_path (str): The file path to the image containing the hidden message.
    key (str): The decryption key used to decrypt the extracted message.

Returns:
    str: The decrypted message extracted from the image.
"""
def lsb_decode(img_path, key):
    img = Image.open(img_path)
    width, height = img.size
    bits = []
    for y in range(height):
        for x in range(width):
            pixel = img.getpixel((x, y))
            for n in range(3):
                bits.append(str(pixel[n] & 1))
    chars = [chr(int(''.join(bits[i:i+8]), 2)) for i in range(0, len(bits), 8)]
    message = ''.join(chars)
    # Extract message up to null-terminator
    enc_b64 = message.split(chr(0))[0]
    # Decode base64 to bytes
    enc_bytes = base64.urlsafe_b64decode(enc_b64)
    # Decrypt
    return decrypt_message(enc_bytes, key)

## Usage example

In [5]:
inputPath = "assets/images/raw.png"
outputPath = "assets/images/lsb-encoded.png"

secret_data = {
    "message": "Very Secret Message",
    "date": datetime.now().isoformat(),
    "location": get_current_location()
}

secret_message = json.dumps(secret_data, indent=4)

key = generate_key()
print("Generated encryption key (store this securely):")
print(key.decode())

# To encode:
lsb_encode(inputPath, secret_message, key, outputPath)

# To decode:
message = lsb_decode(outputPath, key)
print("\nDecoded message:")
print(secret_message)

Generated encryption key (store this securely):
uc8W6tsJyy-Ofkf8m-oylrieSz2uj81dFfj7YHsJqpY=

Decoded message:
{
    "message": "Very Secret Message",
    "date": "2025-06-15T09:22:02.229444",
    "location": {
        "city": "Plessisville",
        "region": "Quebec",
        "country": "CA",
        "coordinates": "46.2186,-71.7620"
    }
}
